## Метрики качества для обучения модели предсказания пространственных связей

**ВАЖНО** это не метрика оценки общего разбора сцены а метрики используемые при обучении модель определять связь между парами объектов

#### Binary F1 (есть связь / нет связи)

```
True label: 1, если target != "нет связи", иначе 0
Predicted label: 1, если predicted_target != "нет связи", иначе 0
Считаем TP, FP, FN по обычной binary классификации
```

#### Strict F1 (точное соответствие триплета, после лемматизации разумеется)

```
Precision/Recall/F1 по триплетам (obj1, obj2, relation)
```

можно придумывать несметное количество разнообразных метрик, но какой в этом смысл? 

In [3]:
examples = [
    {"relation": ["домкрат", "машина"], "target": "под", "predicted_target": "под"},
    {"relation": ["стол", "комната"], "target": "в", "predicted_target": "в"},
    {"relation": ["провод", "книга"], "target": "нет связи", "predicted_target": "нет связи"},
    {"relation": ["лампа", "дверь"], "target": "нет связи", "predicted_target": "нет связи"},
    {"relation": ["вилка", "тарелка"], "target": "на", "predicted_target": "рядом с"},
    {"relation": ["мяч", "пол"], "target": "на", "predicted_target": "в"},
    {"relation": ["диван", "ковёр"], "target": "на", "predicted_target": "нет связи"},
    {"relation": ["стол", "лампа"], "target": "под", "predicted_target": "нет связи"},
    {"relation": ["светильник", "окно"], "target": "нет связи", "predicted_target": "рядом с"},
    {"relation": ["дверь", "стул"], "target": "нет связи", "predicted_target": "на"},
    {"relation": ["телевизор", "тумбочка"], "target": "поверх", "predicted_target": "поверх"},
    {"relation": ["батарея", "стена"], "target": "прикреплён к", "predicted_target": "прикреплён к"},
    {"relation": ["пульт", "телевизор"], "target": "перед", "predicted_target": "рядом с"},
    {"relation": ["вентилятор", "потолок"], "target": "прикреплён к", "predicted_target": "на"},
    {"relation": ["кресло", "стол"], "target": "рядом с", "predicted_target": "рядом с"},
    {"relation": ["бутылка", "стена"], "target": "рядом с", "predicted_target": "рядом с"},
    {"relation": ["шкаф", "коробка"], "target": "в", "predicted_target": "на"},
    {"relation": ["коробка", "шкаф"], "target": "на", "predicted_target": "в"},
    {"relation": ["чайник", "диван"], "target": "нет связи", "predicted_target": "нет связи"},
    {"relation": ["часы", "стена"], "target": "висит на", "predicted_target": "прикреплён к"},
]

In [6]:
def evaluate_relation_predictions(data):
    true_binary = [0 if item["target"] == "нет связи" else 1 for item in data]
    pred_binary = [0 if item["predicted_target"] == "нет связи" else 1 for item in data]

    # Binary F1 вручную чтобы не грузить громоздкий sklearn
    TP = sum(1 for t, p in zip(true_binary, pred_binary) if t == 1 and p == 1)
    FP = sum(1 for t, p in zip(true_binary, pred_binary) if t == 0 and p == 1)
    FN = sum(1 for t, p in zip(true_binary, pred_binary) if t == 1 and p == 0)

    precision_binary = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall_binary = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1_binary = (2 * precision_binary * recall_binary) / (precision_binary + recall_binary) if (precision_binary + recall_binary) > 0 else 0.0

    # Strict F1 (точное совпадение метки связи)
    TP_strict = sum(1 for item in data if item["target"] == item["predicted_target"])
    FP_strict = len(data) - TP_strict  # каждая пара — один шанс на TP
    FN_strict = FP_strict  # аналогично

    precision_strict = TP_strict / (TP_strict + FP_strict) if (TP_strict + FP_strict) > 0 else 0.0
    recall_strict = TP_strict / (TP_strict + FN_strict) if (TP_strict + FN_strict) > 0 else 0.0
    f1_strict = (2 * precision_strict * recall_strict) / (precision_strict + recall_strict) if (precision_strict + recall_strict) > 0 else 0.0

    return {
        "F1binary": round(f1_binary, 4),
        "F1strict": round(f1_strict, 4),
    }



In [7]:
evaluate_relation_predictions(examples)

{'F1binary': 0.8667, 'F1strict': 0.45}